In [ ]:
""" 
This is a notebook with loops and functions to extract all images from the ADNI dataset typical directory tree
after download and organise them in folders according to class. The class data (and more data) is stored in the .xml
file that comes with the metadata .zip, which after extracting is in the parent ADNI directory.


In [ ]:
#Library imports

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as matim
import pylab as pl
import nibabel
import nibabel.processing
import os
import xml.etree.cElementTree as et
from skimage.filters import threshold_otsu
import cc3d
import shutil
import pickle
import pandas as pd

In [2]:
#USE SMALL GPU#
use_gpu = 1 
# The largest memory size GPU is always the first one (0) as they are sorted by size!
gpus=tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_visible_devices(gpus[use_gpu], 'GPU')

In [ ]:
#Create mean .nii in a directory with the exisiting frames

In [ ]:
#Toy example
cd ADNI_Brain_PET__Raw_FDG/2011-06-09_08_23_48.0/I239487/

mean=np.zeros(1)
for file in os.listdir():
    if ".nii" in file:
        print("Name:",file)
        frame=nibabel.load(file).get_fdata()
        print("Frame mean",frame.mean())
        mean=mean+frame
        print("New mean mean",mean.mean())
mean=mean/6
print("Final mean",mean.mean())

In [3]:
#If the means were already created, this deletes them

for root, dirs, files in os.walk("/local_mount/space/celer/1/users/david/Datasets/new_ADNI/ADNI/"):
    for file in files:
        if file=="mean.nii":
            os.remove(os.path.join(root,file))

In [4]:
#Loop for image arrangement in classes after mean.nii creation, AD/MCI/Control example
r=0
mistakes=[]
for root, dirs, files in os.walk("/local_mount/space/celer/1/users/david/Datasets/new_ADNI/ADNI/"):
    for file in files:
        if root=="/local_mount/space/celer/1/users/david/Datasets/new_ADNI/ADNI/":
            print("Base directory, nothing to do here")
        
        elif r!=root:
            print(root)
            os.chdir(root)
            try:
                if len(os.listdir())>2:
                    mean=np.zeros(1)
                    for i in os.listdir(root):
                        if i.endswith(".nii"):
                            #print(i)
                            frame=nibabel.load(i)
                            framearray=frame.get_fdata()
                            #print("Framean",framearray.mean())
                            mean=mean+framearray
                            #print("Meanmean",mean.mean())
                    mean=mean/6
                    mean=mean[:,:,:,0]
                    print("Final mean",mean.mean())
                    nimg=nibabel.Nifti1Image(mean, frame.affine)
                    nibabel.save(nimg,os.path.join(root, "mean.nii"))
                    %cd -q .. 
                    key=os.listdir()[0]
                    #print(key)
            
                    for element in os.listdir("/local_mount/space/celer/1/users/david/Datasets/new_ADNI/ADNI/"):
                        if key in element:
                            #print("Got the key!",element)
                            tree=et.parse(os.path.join("/local_mount/space/celer/1/users/david/Datasets/new_ADNI/ADNI/",element))
                            raiz=tree.getroot()
                            for group in raiz.iter("researchGroup"):
                                a=group.text
                                print("Class is: ",a)
            
                    %cd -q ..
                    %cd -q ..
                    pat=os.getcwd()
                    name=os.path.basename(pat)
            
                    src=os.path.join(root, "mean.nii")
                    pbasedir="/local_mount/space/celer/1/users/notebooks/moises/pdata/newdata"
                    if a=="CN":
                        dst=("{0}/control/{1}_{2}.nii".format(pbasedir,name,key))
                        os.symlink(src,dst)
                        print("Created symlink \n")
                    elif a=="MCI":
                        dst=("{0}/mci/{1}_{2}.nii".format(pbasedir,name,key))
                        os.symlink(src,dst)
                        print("Created symlink \n")
                    elif a=="AD":
                        dst=("{0}/ad/{1}_{2}.nii".format(pbasedir,name,key))
                        os.symlink(src,dst)
                        print("Created symlink \n")
                else:
                    print("No images in this directory")
            except Exception as e:
                print("Error in the pipeline")
                print(e)
                mistakes.append(root)
                
        r=root
        

Base directory, nothing to do here
Base directory, nothing to do here
Base directory, nothing to do here
Base directory, nothing to do here
Base directory, nothing to do here
Base directory, nothing to do here
Base directory, nothing to do here
Base directory, nothing to do here
Base directory, nothing to do here
Base directory, nothing to do here
Base directory, nothing to do here
Base directory, nothing to do here
Base directory, nothing to do here
Base directory, nothing to do here
Base directory, nothing to do here
Base directory, nothing to do here
Base directory, nothing to do here
Base directory, nothing to do here
Base directory, nothing to do here
Base directory, nothing to do here
Base directory, nothing to do here
Base directory, nothing to do here
Base directory, nothing to do here
Base directory, nothing to do here
Base directory, nothing to do here
Base directory, nothing to do here
Base directory, nothing to do here
Base directory, nothing to do here
Base directory, noth

Class is:  CN
Created symlink 

/local_mount/space/celer/1/users/david/Datasets/new_ADNI/ADNI/005_S_0602/30_min_3D_FDG_-_Iter_Brain_Mode/2010-12-17_09_57_02.0/I209110
Final mean 2084.4843699944713
Class is:  CN
Created symlink 

/local_mount/space/celer/1/users/david/Datasets/new_ADNI/ADNI/022_S_6716/ADNI_Brain_PET__Raw/2019-07-24_13_50_10.0/I1214884
Final mean 1914.1033452411466
Class is:  MCI
Created symlink 

/local_mount/space/celer/1/users/david/Datasets/new_ADNI/ADNI/022_S_6716/ADNI_Brain_PET__Raw_FDG/2019-07-24_13_50_10.0/I1214884
No images in this directory
/local_mount/space/celer/1/users/david/Datasets/new_ADNI/ADNI/128_S_5123/ADNI_Brain_PET__Raw_FDG/2013-05-10_11_09_00.0/I372402
Final mean 0.002110818212127513
Class is:  AD
Created symlink 

/local_mount/space/celer/1/users/david/Datasets/new_ADNI/ADNI/099_S_6632/3D_BRAIN_AC/2019-01-18_09_23_59.0/I1117860
Final mean 2363.3135118926098
Class is:  MCI
Created symlink 

/local_mount/space/celer/1/users/david/Datasets/new_ADNI/A

Final mean 1121.5254251013653
Class is:  MCI
Created symlink 

/local_mount/space/celer/1/users/david/Datasets/new_ADNI/ADNI/094_S_4234/30MIN_3D_FDG_4i_16s/2013-11-04_09_40_03.0/I397489
Final mean 795.7976747543094
Class is:  CN
Created symlink 

/local_mount/space/celer/1/users/david/Datasets/new_ADNI/ADNI/094_S_4234/30MIN_3D_FDG_4i_16s/2011-11-04_06_53_07.0/I264710
Final mean 609.0188561495022
Class is:  CN
Created symlink 

/local_mount/space/celer/1/users/david/Datasets/new_ADNI/ADNI/002_S_5018/ADNI_Brain_PET__Raw_FDG/2012-11-14_08_14_11.0/I346569
Final mean 122.23394085919418
Class is:  AD
Created symlink 

/local_mount/space/celer/1/users/david/Datasets/new_ADNI/ADNI/128_S_1408/ADNI_Brain_PET__Raw_FDG/2010-10-28_12_21_00.0/I200599
Final mean 0.0031751696288982658
Class is:  MCI
Created symlink 

/local_mount/space/celer/1/users/david/Datasets/new_ADNI/ADNI/153_S_6694/_FDG/2019-05-03_13_38_09.0/I1162578
No images in this directory
/local_mount/space/celer/1/users/david/Datasets/ne

Final mean 1975.5634413623307
Class is:  MCI
Created symlink 

/local_mount/space/celer/1/users/david/Datasets/new_ADNI/ADNI/041_S_0679/30_min_3D_FDG__4i_16s/2008-08-18_12_30_31.0/I116296
Final mean 1907.9403686523435
Class is:  MCI
Created symlink 

/local_mount/space/celer/1/users/david/Datasets/new_ADNI/ADNI/041_S_0679/30_min_3D_FDG__4i_16s/2006-08-04_07_08_28.0/I20162
Final mean 1758.646611612309
Class is:  MCI
Created symlink 

/local_mount/space/celer/1/users/david/Datasets/new_ADNI/ADNI/041_S_0679/30_min_3D_FDG__4i_16s/2009-07-23_11_00_07.0/I150289
Final mean 1931.9381093827508
Class is:  MCI
Created symlink 

/local_mount/space/celer/1/users/david/Datasets/new_ADNI/ADNI/036_S_5063/30_MIN_3D_FDG_4i_16s/2013-02-11_10_51_38.0/I358620
Final mean 822.1118524137632
Class is:  AD
Created symlink 

/local_mount/space/celer/1/users/david/Datasets/new_ADNI/ADNI/135_S_6389/Dyn_Brain_VPHD_6i16s_AC/2018-07-02_09_21_57.0/I1016219
Final mean 2017.4539982985868
Class is:  AD
Created symlink 



Final mean 1028.767521931563
Class is:  CN
Created symlink 

/local_mount/space/celer/1/users/david/Datasets/new_ADNI/ADNI/035_S_4082/ADNI-FDG_BRAIN__AC_/2011-07-05_14_39_21.0/I244384
Final mean 1240.8826020580368
Class is:  CN
Created symlink 

/local_mount/space/celer/1/users/david/Datasets/new_ADNI/ADNI/035_S_4082/ADNI-FDG_BRAIN__AC_/2013-07-09_11_50_13.0/I380745
Final mean 967.7182925032182
Class is:  CN
Created symlink 

/local_mount/space/celer/1/users/david/Datasets/new_ADNI/ADNI/941_S_4292/ADNI_Brain_PET__Raw_FDG/2013-10-09_13_19_48.0/I393814
Final mean 107.03365444607209
Class is:  CN
Created symlink 

/local_mount/space/celer/1/users/david/Datasets/new_ADNI/ADNI/941_S_4292/ADNI_Brain_PET__Raw_FDG/2012-01-13_10_57_14.0/I277675
Final mean 133.63641413935906
Class is:  CN
Created symlink 

/local_mount/space/celer/1/users/david/Datasets/new_ADNI/ADNI/013_S_1161/ADNI_STATIC_BRAIN__6X5_/2007-07-03_10_22_23.0/I58721
Final mean 37.534189034696325
Class is:  AD
Created symlink 

/loc

...

Final mean 2209.3077906527965
Class is:  CN
Created symlink 

/local_mount/space/celer/1/users/david/Datasets/new_ADNI/ADNI/098_S_4018/ANONYMIZED_FDG/2011-06-01_09_38_50.0/I238478
Final mean 2106.822804878385
Class is:  CN
Created symlink 

/local_mount/space/celer/1/users/david/Datasets/new_ADNI/ADNI/130_S_4660/ADNI_FDG_Brain_30min_dyn/2012-05-11_10_31_01.0/I303026
Final mean 737.2705326244378
Class is:  AD
Created symlink 

/local_mount/space/celer/1/users/david/Datasets/new_ADNI/ADNI/130_S_4660/ADNI_FDG_Brain_30min_dyn/2014-05-07_09_48_03.0/I424025
Final mean 600.1489674480069
Class is:  AD
Created symlink 

/local_mount/space/celer/1/users/david/Datasets/new_ADNI/ADNI/041_S_1260/30_min_3D_FDG__4i_16s/2007-03-08_07_51_37.0/I43223
Final mean 1270.809937330781
Class is:  MCI
Created symlink 

/local_mount/space/celer/1/users/david/Datasets/new_ADNI/ADNI/041_S_1260/30_min_3D_FDG__4i_16s/2008-09-15_10_26_34.0/I117960
Final mean 1892.765159485832
Class is:  MCI
Created symlink 

/local_m

Final mean 0.004119056230451403
Class is:  CN
Created symlink 

/local_mount/space/celer/1/users/david/Datasets/new_ADNI/ADNI/032_S_4429/ADNI_Brain_PET__Raw_FDG/2014-01-21_12_34_22.0/I407299
Final mean 0.002660709294496309
Class is:  CN
Created symlink 

/local_mount/space/celer/1/users/david/Datasets/new_ADNI/ADNI/032_S_1169/ADNI_Brain_PET__Raw/2017-12-21_09_47_06.0/I948825
Final mean 0.003233606508704425
Class is:  CN
Created symlink 

/local_mount/space/celer/1/users/david/Datasets/new_ADNI/ADNI/032_S_1169/ADNI_Brain_PET__Raw_FDG/2017-12-21_09_47_06.0/I948825
No images in this directory
/local_mount/space/celer/1/users/david/Datasets/new_ADNI/ADNI/032_S_1169/ADNI_Brain_PET__Raw_FDG/2013-02-14_12_19_45.0/I361163
Final mean 0.002819206423058257
Class is:  CN
Created symlink 

/local_mount/space/celer/1/users/david/Datasets/new_ADNI/ADNI/032_S_1169/ADNI_Brain_PET__Raw_FDG/2011-01-19_14_03_53.0/I213930
Final mean 0.004202656105905306
Class is:  CN
Created symlink 

/local_mount/space/ce

Final mean 1494.1768015327186
Class is:  MCI
Created symlink 

/local_mount/space/celer/1/users/david/Datasets/new_ADNI/ADNI/021_S_0626/ADNI_STATIC_6X5/2009-08-12_10_25_15.0/I152992
Final mean 2407.1821473580753
Class is:  MCI
Created symlink 

/local_mount/space/celer/1/users/david/Datasets/new_ADNI/ADNI/021_S_0626/ADNI_STATIC_6X5/2008-08-27_09_48_59.0/I117146
Final mean 2149.7993247524446
Class is:  MCI
Created symlink 

/local_mount/space/celer/1/users/david/Datasets/new_ADNI/ADNI/021_S_0626/ADNI_FDG_STATIC_6X5/2010-08-17_11_05_49.0/I191833
Final mean 2404.261327130296
Class is:  MCI
Created symlink 

/local_mount/space/celer/1/users/david/Datasets/new_ADNI/ADNI/021_S_0626/3DFDG-BRAINSCAN-DYNEMISSION/2007-02-15_10_11_59.0/I40178
Final mean 2131.209252104442
Class is:  MCI
Created symlink 

/local_mount/space/celer/1/users/david/Datasets/new_ADNI/ADNI/021_S_0626/ADNISTATIC6X5/2008-02-12_10_05_17.0/I90202
Final mean 1962.0657936408936
Class is:  MCI
Created symlink 

/local_mount/spac

Final mean 858.8282748429239
Class is:  AD
Created symlink 

/local_mount/space/celer/1/users/david/Datasets/new_ADNI/ADNI/127_S_6549/30-60_min_dynamic-3D_4i_16s_FDG/2018-10-08_10_50_08.0/I1061154
No images in this directory
/local_mount/space/celer/1/users/david/Datasets/new_ADNI/ADNI/002_S_0685/ADNI_Brain_PET__Raw_FDG/2010-07-27_08_37_48.0/I187652
Final mean 73.31215628164786
Class is:  CN
Created symlink 

/local_mount/space/celer/1/users/david/Datasets/new_ADNI/ADNI/002_S_0685/ADNI_Brain_PET__Raw_FDG/2012-08-02_08_37_38.0/I321228
Final mean 132.69166451913338
Class is:  CN
Created symlink 

/local_mount/space/celer/1/users/david/Datasets/new_ADNI/ADNI/127_S_4645/30-60_min_dynamic-3D_-_Iter__FDG/2014-04-08_08_03_09.0/I419533
Final mean 585.7649700830856
Class is:  CN
Created symlink 

/local_mount/space/celer/1/users/david/Datasets/new_ADNI/ADNI/127_S_4645/30-60_min_dynamic-3D_4i_16s__FDG/2012-04-19_11_46_40.0/I298693
Final mean 516.8225157399656
Class is:  CN
Created symlink 

/loc

Final mean 1591.5940328827994
Class is:  CN
Created symlink 

/local_mount/space/celer/1/users/david/Datasets/new_ADNI/ADNI/009_S_0751/30_min_3D_FDG_6i_16s/2006-08-21_11_33_18.0/I23216
Final mean 1718.7900100599788
Class is:  CN
Created symlink 

/local_mount/space/celer/1/users/david/Datasets/new_ADNI/ADNI/022_S_4320/ADNI_Brain_PET__Raw_FDG/2013-12-13_12_34_57.0/I403215
Final mean 1811.3505126493692
Class is:  CN
Created symlink 

/local_mount/space/celer/1/users/david/Datasets/new_ADNI/ADNI/022_S_4320/ADNI_Brain_PET__Raw_FDG/2011-12-09_14_08_32.0/I273298
Final mean 1501.8647429740613
Class is:  CN
Created symlink 

/local_mount/space/celer/1/users/david/Datasets/new_ADNI/ADNI/941_S_6803/ADNI3-BRAIN/2019-10-07_15_05_36.0/I1238836
Final mean 2388.05364944453
Class is:  MCI
Created symlink 

/local_mount/space/celer/1/users/david/Datasets/new_ADNI/ADNI/941_S_6803/ADNI3-BRAIN_FDG/2019-10-07_15_05_36.0/I1238836
No images in this directory
/local_mount/space/celer/1/users/david/Datasets/ne

In [ ]:
#Loop for image arrangement without mean.nii creation, meaning we use all separate frames.

r=0
mistakes=[]
for root, dirs, files in os.walk("/local_mount/space/celer/1/users/david/Datasets/new_ADNI/ADNI/"):
    for file in files:
        if root=="/local_mount/space/celer/1/users/david/Datasets/new_ADNI/ADNI/":
            print("Base directory, nothing to do here")
        
        elif r!=root:
            print(root)
            os.chdir(root)
            try:
                if len(os.listdir())>1:
                    %cd -q .. 
                    key=os.listdir()[0]
                    #print(key)
                    %cd -q ..
                    %cd -q ..
                    pat=os.getcwd()
                    name=os.path.basename(pat)
                                           
                    for element in os.listdir("/local_mount/space/celer/1/users/david/Datasets/new_ADNI/ADNI/"):
                        if key in element:
                            #print("Got the key!",element)
                            tree=et.parse(os.path.join("/local_mount/space/celer/1/users/david/Datasets/new_ADNI/ADNI/",element))
                            raiz=tree.getroot()
                            for group in raiz.iter("researchGroup"):
                                clas=group.text
                                print("Class is: ",clas)
                    fc=0            
                    for i in os.listdir(root):
                        if i.endswith(".nii"):
                            src=os.path.join(root, "mean.nii")
                            pbasedir="/local_mount/space/celer/1/users/notebooks/moises/pdata/framedata"
                            if clas=="CN":
                                dst=("{0}/control/{1}_{2}_{3}.nii".format(pbasedir,name,key,fc))
                                os.symlink(src,dst)
                                print("Created symlink \n")
                            elif clas=="MCI":
                                dst=("{0}/mci/{1}_{2}_{3}.nii".format(pbasedir,name,key,fc))
                                os.symlink(src,dst)
                                print("Created symlink \n")
                            elif clas=="AD":
                                dst=("{0}/ad/{1}_{2}_{3}.nii".format(pbasedir,name,key,fc))
                                os.symlink(src,dst)
                                print("Created symlink \n")
                            else:
                                print("Problem")
                            fc=fc+1
                    if fc<5:
                        print("This set has less than 6 frames")
                else:
                    print("No images in this directory")
            except Exception as e:
                print("Error in the pipeline")
                print(e)
                mistakes.append(root)
                
        r=root
        